# Task 4: Cars-196

### Mounting Google drive

In [1]:
# Mounting Google drive for loading the checkpoint.

from google.colab import drive
drive.mount('/content/gdrive')

!cd gdrive/MyDrive/

Mounted at /content/gdrive


### Import Dependencies

In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from scipy import io
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor, Lambda
import torchvision.models as models
from torch.hub import load_state_dict_from_url


torch.cuda.empty_cache()
cuda = torch.device('cuda') 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


### 1. Preprocessing: Collate all image's path address & labels as a input to custom pytorch dataset

### 2. Custom Pytorch datatset for training images of "Cars-196" with first 96 classes dataset.

### 3. Custom Pytorch datatset for testing images of "Cars-196" with rest of 96 classes dataset.

In [3]:
file_list_train = []
file_list_test = []

labels_train = {}
labels_test = {}

mtf = io.loadmat('gdrive/MyDrive/cars/cars_train_annos.mat')

for idx, img_ in enumerate(mtf['annotations'][0]):

  if img_[4][0][0] <= 98:
    dir = 'gdrive/MyDrive/cars/cars_train/' + img_[5][0]
    labels_train[dir] = img_[4][0][0] - 1
    file_list_train.append((dir))
  else:
    dir = 'gdrive/MyDrive/cars/cars_train/' + img_[5][0]
    labels_test[dir] = img_[4][0][0] - 1 -98
    file_list_test.append((dir))


class TrainCarsDataset(Dataset):
    def __init__(self, f_list, id, transform=None):

        self.filenames = f_list
        self.transform = transform
        self.id_dict = id

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):

        img_path = self.filenames[idx]
        image = None
      
        with open(img_path, 'rb') as f:
          image = Image.open(f)
          image =  image.convert('RGB')
          
        
        
        label = self.id_dict[img_path]
       

        if self.transform is not None:

            image = self.transform(image)
            
        return image, label


class TrainSet(TrainCarsDataset):
  
    def __init__(self, f_list, id, transform=None):

        super(TrainSet, self).__init__(f_list, id, transform=transform)


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
trans = transforms.Compose([
            transforms.Resize(size=(64, 64)),
            #transforms.Resize(size=(224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])

trainset = TrainSet(f_list=file_list_train,id=labels_train, transform=trans)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=4)


class TestCarsDataset(Dataset):
    def __init__(self, f_list, id,transform=None):
        self.filenames = f_list
        self.transform = transform
        self.id_dict = id
    
    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_path = self.filenames[idx]
        image = None
  
        with open(img_path, 'rb') as f:
          image = Image.open(f)
          image =  image.convert('RGB')
        
        label = self.id_dict[img_path]
        
        if self.transform is not None:
            image = self.transform(image)
        return image, label


trans_test = transforms.Compose([
            transforms.Resize(size=(64, 64)),                     
            #transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            normalize,
        ])


testset = TestCarsDataset(f_list=file_list_test,id=labels_test, transform=trans_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=256, shuffle=False, num_workers=4)


### CUBS Block 1

In [4]:
class CUBS1(nn.Module):
  def __init__(self, channels_in, n_in):
      super(CUBS1, self).__init__()
      self.channels_in = channels_in
      self.n_in = n_in
      self.dense_1_1 = nn.Linear(channels_in,n_in)
      self.dense_1_2 = nn.Linear(channels_in,n_in)
      self.dense_1_3 = nn.Linear(channels_in,n_in)
      self.dense_2 = nn.Linear(n_in,channels_in)
      self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    

      self.relu = nn.ReLU(inplace=True)
      self.sig = nn.Sigmoid()


  def CB1_Sim1(self, a, b):
    a_norm = torch.nn.functional.normalize(a, dim=2)
    b_norm = torch.nn.functional.normalize(b, dim=2)

    sim_matrix = torch.matmul(a_norm[0].T, b_norm[0])
    sim_matrix = sim_matrix.reshape((1,self.n_in,self.n_in))


    for i in range(1,a.shape[0]):
  
      mul = torch.matmul(a_norm[i].T, b_norm[i]) 
      mul = mul.reshape((1,self.n_in,self.n_in))
      sim_matrix = torch.vstack((sim_matrix,mul))
      

    

    return sim_matrix


  def CB1_Sim2(self, a, b):
    b_norm = torch.nn.functional.normalize(b, dim=2)

    sim_matrix = torch.matmul(a[0], b_norm[0].T)
    sim_matrix = sim_matrix.reshape((1,1,self.n_in))


    for i in range(1,a.shape[0]):
      mul = torch.matmul(a[i], b_norm[i].T)
      mul = mul.reshape((1,1,self.n_in))
      sim_matrix = torch.vstack((sim_matrix,mul))

    return sim_matrix

  def CB1_Softmax(self, a):
    out_matrix = nn.Softmax(dim=1)(a[0])
    out_matrix = out_matrix.reshape((1,self.n_in,self.n_in))

    for i in range(1,a.shape[0]):
      sim = nn.Softmax(dim=1)(a[i])
      sim = sim.reshape((1,self.n_in,self.n_in))
      out_matrix = torch.vstack((out_matrix,sim))

    return out_matrix

  def CB1_Channel(self, a, b):
    out_matrix = b[0] * a[0].unsqueeze(dim=-1). unsqueeze(dim=-1)

    for i in range(1,a.shape[0]): 
      mul = b[i] * a[i].unsqueeze(dim=-1). unsqueeze(dim=-1)
      out_matrix = torch.vstack((out_matrix,mul))

    return out_matrix

      
  

  def forward(self, x):
      x_pool = self.avgpool(x)
      x_pool = torch.reshape(x_pool,(x.shape[0],1,x.shape[1]))

     

      x_1_1 = self.dense_1_1(x_pool)
     
      
      x_1_1 = self.relu(x_1_1)

      x_1_2 = self.dense_1_2(x_pool)
      x_1_2 = self.relu(x_1_2)

      x_1_3 = self.dense_1_3(x_pool)
      x_1_3 = self.relu(x_1_3)

     
      x_1_2_sim = self.CB1_Sim1(x_1_1, x_1_2)


            
      x_1_2_softmax = self.CB1_Softmax(x_1_2_sim)

  
      x_1_2_3_sim = self.CB1_Sim2(x_1_3, x_1_2_softmax)
      
      x_2 = self.dense_2(x_1_2_3_sim)
      x_2 = self.relu(x_2)

      x_concat_1 = torch.add(x_2, x_pool)
      x_sig = self.sig(x_concat_1)


      x_out = self.CB1_Channel(x_sig, x)
      return x_out



### CUBS Block 2

In [5]:

class CUBS2(nn.Module):
  def __init__(self, channels_in, n_in):
      super(CUBS2, self).__init__()
      self.channels_in = channels_in
      self.n_in = n_in


      self.conv_1_1 = nn.Conv2d(channels_in, 1, kernel_size=1)
      self.conv_1_2 = nn.Conv2d(channels_in, 1, kernel_size=1)
      self.conv_1_3 = nn.Conv2d(channels_in, 1, kernel_size=1)
      
   
      
      self.relu = nn.ReLU(inplace=True)

      self.sig = nn.Sigmoid()


  def CB2_Sim1(self, a, b):

      a = torch.flatten(a, start_dim=2, end_dim=3)    
      b = torch.flatten(b, start_dim=2, end_dim=3)

      a_norm = torch.nn.functional.normalize(a, dim=2)
      b_norm = torch.nn.functional.normalize(b, dim=2)

      sim_matrix = torch.matmul(a_norm[0].T, b_norm[0])
      sim_matrix = sim_matrix.reshape((1,a.shape[2],a.shape[2]))


      for i in range(1,a.shape[0]):
    
        mul = torch.matmul(a_norm[i].T, b_norm[i]) 
        mul = mul.reshape((1,a.shape[2],a.shape[2]))
        sim_matrix = torch.vstack((sim_matrix,mul))
        

      return sim_matrix


  def CB2_Sim2(self, a, b):

      a = torch.flatten(a, start_dim=2, end_dim=3)
      b_norm = torch.nn.functional.normalize(b, dim=2)

      sim_matrix = torch.matmul(a[0], b_norm[0].T)
      sim_matrix = sim_matrix.reshape((1,1,a.shape[2]))


      for i in range(1,a.shape[0]):
        mul = torch.matmul(a[i], b_norm[i].T)
        mul = mul.reshape((1,1,a.shape[2]))
        sim_matrix = torch.vstack((sim_matrix,mul))

      return sim_matrix


  def CB2_Softmax(self, a):
      out_matrix = nn.Softmax(dim=1)(a[0])
      out_matrix = out_matrix.reshape((1,a.shape[2],a.shape[2]))

      for i in range(1,a.shape[0]):
        sim = nn.Softmax(dim=1)(a[i])
        sim = sim.reshape((1,a.shape[2],a.shape[2]))
        out_matrix = torch.vstack((out_matrix,sim))

      return out_matrix

  

  def bmul(self, vec, mat, axis=0):
      mat = mat.transpose(axis, -1)
      return (mat * vec.expand_as(mat)).transpose(axis, -1)

  def CB1_Pixel(self, a, b):
      a = torch.reshape(a, (a.shape[0],b.shape[2],b.shape[3]))
    
      a_0 = a[0]
      b_0 = b[0]
      out_matrix = self.bmul(a_0,b_0, axis=2)
      out_matrix = out_matrix.reshape((1,b.shape[1],b.shape[2],b.shape[3]))

      for i in range(1, a.shape[0]):
        ai = a[i]
        
        bi = b[i]
      
        mul = self.bmul(ai,bi, axis=2)
        mul = mul.reshape((1,b.shape[1],b.shape[2],b.shape[3]))
        out_matrix = torch.vstack((out_matrix,mul))

      return out_matrix
      
  def forward(self, x):

     
      x_1_1 = self.conv_1_1(x)
      x_1_1 = self.relu(x_1_1)

      x_1_2 = self.conv_1_2(x)
      x_1_2 = self.relu(x_1_2)

      x_1_3 = self.conv_1_3(x)
      x_1_3 = self.relu(x_1_3)

     
      x_1_2_sim = self.CB2_Sim1(x_1_1, x_1_2)

      x_1_2_softmax = self.CB2_Softmax(x_1_2_sim)

     
      x_1_2_3_sim = self.CB2_Sim2(x_1_3, x_1_2_softmax)
      
      x_sig = self.sig(x_1_2_3_sim)

     
      x_out = self.CB1_Pixel(x_sig, x)
      
      return x_out


### This section contains Open-Sourced "ResNet-18" implementation along with various modifications and CUBS Blocks(1 & 2) experiments mentioned in the assignment.

***

### I've used "BasicBlock_C" from Task-3 as it provided me with best accuracy on testing dataset.

***

### BasicBlock_C : CUBS 2 -> CUBS 1



In [6]:
__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
           'wide_resnet50_2', 'wide_resnet101_2']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)



class BasicBlock_B(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock_B, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride
        self.cubs1 = CUBS1(planes, 30)
        self.cubs2 = CUBS2(planes, 30)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        # CUBS 1 -> CUBS 2
        out = self.cubs1(out)
        out = self.cubs2(out)

        out += identity
        out = self.relu(out)

        return out


class BasicBlock_C(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock_C, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride
        self.cubs1 = CUBS1(planes, 30)
        self.cubs2 = CUBS2(planes, 30)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        # CUBS 2 -> CUBS 1
        out = self.cubs2(out)
        out = self.cubs1(out)

        out += identity
        out = self.relu(out)

        return out



class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=200, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model




def model_B(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock_B, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)
    

def model_C(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock_C, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)


### Defining Training and Testing Function

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
      
        X, y = X.to(device), y.long().to(device)
        optimizer.zero_grad()
        torch.set_grad_enabled(True)
        
        pred = model(X).float()
       
        loss = loss_fn(pred, y)

       
        loss.backward()
        optimizer.step()

    
        if batch % 4 == 0:
            print("Loss for the batch: ", loss)


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
          
            X, y = X.to(device), y.long().to(device)
            torch.set_grad_enabled(False)
            pred = model(X).float()
       
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

### Defining "Learning Rate Scheduler" & "Early stopping mechanism"

In [8]:
class LRScheduler():
  
    def __init__(
        self, optimizer, patience=5, min_lr=1e-7, factor=0.75
    ):
        
        self.optimizer = optimizer
        self.patience = patience
        self.min_lr = min_lr
        self.factor = factor
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau( 
                self.optimizer,
                mode='min',
                patience=self.patience,
                factor=self.factor,
                min_lr=self.min_lr,
                verbose=True
            )
    def __call__(self, val_loss):
        self.scheduler.step(val_loss)


class EarlyStopping():
   
    def __init__(self, patience=5, min_delta=0):
      
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

### Instantiating & training & checkpoint "Model_C"

#### There are 2 cells for "Model_C" that are given below.
#### Run the 1st cell in case you want to load a checkpoint & resume training.
#### Else Run the 2nd cell in case you want to start training from scratch.

#### **NOTE: Make sure to change the path of loading the checkpoint in 1st cell**

In [9]:
model_C = model_C(pretrained=False, progress=True)
model_C = model_C.to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer_model_C = torch.optim.Adam(model_C.parameters(), lr=0.0001)


# **NOTE: Make sure to change the path of loading the checkpoint in 1st cell**
checkpoint = torch.load('gdrive/MyDrive/trained-model_C-model.ckpt')


model_C.load_state_dict(checkpoint['model_state_dict'])
optimizer_model_C.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch'] + 1
loss = checkpoint['loss']

#es =  EarlyStopping()
#lrs = LRScheduler(optimizer_model_C)

checkpoint_dir = "."
test_loss = 0
epochs = 10
for i in range(epochs):
    print(f"Epoch {i+1}")
    train(trainloader, model_C, loss_fn, optimizer_model_C)
    #test_loss = test(testloader, model_C, loss_fn)

    torch.save({
        'epoch': i+1,
        'model_state_dict': model_C.state_dict(),
        'optimizer_state_dict': optimizer_model_C.state_dict(),
        'loss': test_loss
        }, checkpoint_dir+'/%04d-model_C-cars.ckpt' %i)
    
    #lrs(test_loss)
    #es(test_loss)
    #if es.early_stop: 
    #     break



Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Loss for the batch:  tensor(11.2441, device='cuda:0', grad_fn=<NllLossBackward>)
Loss for the batch:  tensor(11.3082, device='cuda:0', grad_fn=<NllLossBackward>)
Loss for the batch:  tensor(10.2530, device='cuda:0', grad_fn=<NllLossBackward>)
Loss for the batch:  tensor(10.1868, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch 2
Loss for the batch:  tensor(9.1234, device='cuda:0', grad_fn=<NllLossBackward>)
Loss for the batch:  tensor(8.4102, device='cuda:0', grad_fn=<NllLossBackward>)
Loss for the batch:  tensor(8.5170, device='cuda:0', grad_fn=<NllLossBackward>)
Loss for the batch:  tensor(8.0134, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch 3
Loss for the batch:  tensor(7.4658, device='cuda:0', grad_fn=<NllLossBackward>)
Loss for the batch:  tensor(7.3754, device='cuda:0', grad_fn=<NllLossBackward>)
Loss for the batch:  tensor(6.8968, device='cuda:0', grad_fn=<NllLossBackward>)
Loss for the batch:  tensor(7.0656, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch 4
Loss for the

#### Else Run the 2nd cell in case you want to start training from scratch.

In [ ]:
model_C = model_C(pretrained=False, progress=True)
model_C = model_C.to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer_model_C = torch.optim.Adam(model_C.parameters(), lr=0.0001)

#es =  EarlyStopping()
#lrs = LRScheduler(optimizer_model_C)

checkpoint_dir = "."
test_loss = 0
epochs = 20
for i in range(epochs):
    print(f"Epoch {i+1}")
    train(trainloader, model_C, loss_fn, optimizer_model_C)
    #test_loss = test(testloader, model_C, loss_fn)
    
    torch.save({
        'epoch': i+1,
        'model_state_dict': model_C.state_dict(),
        'optimizer_state_dict': optimizer_model_C.state_dict(),
        'loss': test_loss
        }, checkpoint_dir+'/%04d-model_C-cars.ckpt' %i)
    
    #lrs(test_loss)
    #es(test_loss)
    #if es.early_stop:
    #  break


Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


loss: 5.467802  [    0/ 4052]
Epoch 2
loss: 4.352901  [    0/ 4052]
Epoch 3
loss: 3.913038  [    0/ 4052]
Epoch 4
loss: 4.234994  [    0/ 4052]
Epoch 5
loss: 3.279977  [    0/ 4052]
Epoch 6
loss: 2.858897  [    0/ 4052]
Epoch 7
loss: 3.137208  [    0/ 4052]
Epoch 8
loss: 2.361117  [    0/ 4052]
Epoch 9
loss: 1.805948  [    0/ 4052]
Epoch 10
loss: 1.088871  [    0/ 4052]
Epoch 11
loss: 0.942495  [    0/ 4052]
Epoch 12
loss: 0.434313  [    0/ 4052]
Epoch 13
loss: 0.284304  [    0/ 4052]
Epoch 14
loss: 0.723998  [    0/ 4052]
Epoch 15
loss: 0.388277  [    0/ 4052]
Epoch 16
loss: 0.229776  [    0/ 4052]
Epoch 17
loss: 0.204802  [    0/ 4052]
Epoch 18
loss: 0.344304  [    0/ 4052]
Epoch 19
loss: 0.327055  [    0/ 4052]
Epoch 20
loss: 0.389437  [    0/ 4052]
Test Error: 
 Accuracy: 98.9%, Avg loss: 0.043564 



### Defining a new model that is a copy of previously trained model, but with last classification layer chopped off.

In [10]:
new_model = torch.nn.Sequential(*(list(model_C.children())[:-1]))

### Defining "eval" function to get "512 feature embeddings (predictions)" for whole dataset

In [11]:
def eval(dataloader, model, loss_fn):
   
    model.eval()
    predictions = None
    labels = None
  
    with torch.no_grad():
        for X, y in dataloader:
            
            X, y = X.to(device), y.long().to(device)
            torch.set_grad_enabled(False)
            pred = model(X).float()
            pred = pred.reshape((pred.shape[0],pred.shape[1]))

            if predictions == None:
              predictions = pred
              labels = y
            else:
              predictions = torch.vstack((predictions,pred))
              labels = torch.hstack((labels,y))
        
            

    return predictions, labels
    

### Defining "similarity" fucntion to calculate Similarity-matrix between the "512 feature embeddings (predictions)" for whole dataset/batch

In [15]:
def similarity(preds):
      preds_T = preds.T

      preds_norm = torch.linalg.norm(preds, dim=1, keepdim=True)  
      preds_T_norm = torch.linalg.norm(preds_T, dim=0, keepdim=True) 

      sim = ((preds @ preds_T) / (preds_norm @ preds_T_norm)).T
      return sim

### Defining a function to calculate "Recall @ k" for the given dataset/batch

In [13]:
# Recall@k

def recall_k(sim,y,k):
  
  recall = 0
  for i in range(sim.shape[0]):

    sim_i_list = sim[i]

    sorted_ind = torch.argsort(sim_i_list, descending=True)
    sorted_ind = sorted_ind[:k+1]


    recall_cnt = 0
    for j in sorted_ind:
     
      if y[j] == y[i] and j != i:
        
        recall_cnt +=1


    recall_cnt = recall_cnt/k

    recall += recall_cnt



  recall = recall / sim.shape[0]

  return recall

### Calculate "Recall @K" values for the finetuned model. 

In [16]:
out, y = eval(testloader, new_model, loss_fn) 
sim1 = similarity(out)
rc1 = recall_k(sim1,y,1)
rc2 = recall_k(sim1,y,2)
rc4 = recall_k(sim1,y,4)
rc8 = recall_k(sim1,y,8)

print("Recall at 1: ",rc1)
print("Recall at 2: ",rc2)
print("Recall at 4: ",rc4)
print("Recall at 8: ",rc8)

Recall at 1:  0.0747800586510264
Recall at 2:  0.060117302052785926
Recall at 4:  0.047653958944281524
Recall at 8:  0.038581378299120235


### Defining Custom Loss to Re-train the "new_model"
### **Note: "util" function is used to mine "Hard Negative/Positive Samples"**

In [17]:
class CustomLoss(nn.Module):

    def __init__(self):
        
        super(CustomLoss, self).__init__()      
        self.loss = 0

    def forward(self, output, target):
        self.loss = self.custom_loss(output, target)
        return self.loss


    def similarity(self, preds):
      preds_T = preds.T

      preds_norm = torch.linalg.norm(preds, dim=1, keepdim=True)  
      preds_T_norm = torch.linalg.norm(preds_T, dim=0, keepdim=True) 

      sim = ((preds @ preds_T) / (preds_norm @ preds_T_norm)).T
      return sim


    # Function to select Hard Negative/Positive Mining sets
    def util(self, sim, out,y):
      hard_p = {}
      hard_n = {}

      for lab_i, lab in enumerate(y):
        hard_p[lab_i] = []
        hard_n[lab_i] = []



      for ind_i,i in enumerate(sim):

        sim_i_list = i
        

        sorted_ind = torch.argsort(sim_i_list, descending=True)

        # Hard Negative/Positive Mining threshold
        k = int(sorted_ind.shape[0]/2)

        hard_p_prospective = sorted_ind[-k:]
        hard_n_prospective = sorted_ind[:k]

  
        for k in hard_p_prospective:
          if ind_i == k:
            continue
          elif y[ind_i] == y[k]:
            hard_p[ind_i].append(k)

        for k in hard_n_prospective:
          if ind_i == k:
            continue
          elif y[ind_i] != y[k]:
            hard_n[ind_i].append(k)
            

            

      return hard_p, hard_n



    # Custom Loss Formula
    def loss_formula(self, batch, sim, p, n, alpha=5, beta=50, lamb=0.5):


      psum_batch = 0
      nsum_batch = 0

      for index in range(batch):

        nsum = 0
        for i in n[index]:
          exp_pow = beta*(sim[index][i] - lamb)
          nsum += torch.exp(exp_pow)

        nsum_batch += (1/beta)*(1 + nsum)
      

        psum = 0
        for i in p[index]:
          exp_pow = -alpha*(sim[index][i] - lamb)
          psum += torch.exp(exp_pow)

        psum_batch += (1/alpha)*(1 + psum)



      loss = (psum_batch + nsum_batch)/batch

      return loss


    def custom_loss(self, output, target):
        
        sim = self.similarity(output)
        p, n = self.util(sim,output,target)

        batch =output.shape[0]

        loss = self.loss_formula(batch, sim, p, n)
        
        return loss

### Defining a new training function for the new "CustomLoss"

In [18]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
      
        X, y = X.to(device), y.long().to(device)
        optimizer.zero_grad()
        torch.set_grad_enabled(True)
        
        pred = model(X).float()
        pred = pred.reshape((pred.shape[0],pred.shape[1]))
        loss = loss_fn(pred, y)

       
        loss.backward()
        optimizer.step()

    
        print("Loss for the Batch",loss)

### Re-train the "new_model" with the "CustomLoss"

In [19]:
checkpoint_dir = "."

cust_loss = CustomLoss()
test_loss = 0
epochs = 10
optimizer_new_model = torch.optim.Adam(new_model.parameters(), lr=0.0001)
for i in range(epochs):
    print("Epoch: ", i)
    train(trainloader, new_model, cust_loss, optimizer_new_model)
    torch.save({
        'epoch': i+1,
        'model_state_dict': new_model.state_dict(),
        'optimizer_state_dict': optimizer_new_model.state_dict(),
        'loss': test_loss
        }, checkpoint_dir+'/%04d-new_model-cars-custom-loss.ckpt' %i)


Epoch:  0
Loss for the Batch tensor(59889952., device='cuda:0', grad_fn=<DivBackward0>)
Loss for the Batch tensor(38932596., device='cuda:0', grad_fn=<DivBackward0>)
Loss for the Batch tensor(17139378., device='cuda:0', grad_fn=<DivBackward0>)
Loss for the Batch tensor(9905732., device='cuda:0', grad_fn=<DivBackward0>)
Loss for the Batch tensor(8641176., device='cuda:0', grad_fn=<DivBackward0>)
Loss for the Batch tensor(3913825., device='cuda:0', grad_fn=<DivBackward0>)
Loss for the Batch tensor(4709531., device='cuda:0', grad_fn=<DivBackward0>)
Loss for the Batch tensor(4082372.2500, device='cuda:0', grad_fn=<DivBackward0>)
Loss for the Batch tensor(2158048.5000, device='cuda:0', grad_fn=<DivBackward0>)
Loss for the Batch tensor(4107710., device='cuda:0', grad_fn=<DivBackward0>)
Loss for the Batch tensor(3302969.2500, device='cuda:0', grad_fn=<DivBackward0>)
Loss for the Batch tensor(1870881.1250, device='cuda:0', grad_fn=<DivBackward0>)
Loss for the Batch tensor(1136326.3750, device=

### Calculate Final "Recall @K" values for the re-trained model. 

In [20]:
out, y = eval(testloader, new_model, loss_fn) 
sim1 = similarity(out)
rc1 = recall_k(sim1,y,1)
rc2 = recall_k(sim1,y,2)
rc4 = recall_k(sim1,y,4)
rc8 = recall_k(sim1,y,8)

print("Recall at 1: ",rc1)
print("Recall at 2: ",rc2)
print("Recall at 4: ",rc4)
print("Recall at 8: ",rc8)

Recall at 1:  0.08504398826979472
Recall at 2:  0.06867057673509286
Recall at 4:  0.05510752688172043
Recall at 8:  0.04536290322580645
